In [ ]:
!pip install dipy

In [ ]:
from dipy.data import read_stanford_labels

hardi_img, gtab, labels_img = read_stanford_labels()
data = hardi_img.get_data()
labels = labels_img.get_data()
affine = hardi_img.affine

In [ ]:
white_matter = (labels == 1) | (labels == 2)

In [ ]:
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model

csa_model = CsaOdfModel(gtab, sh_order=6)
csa_peaks = peaks_from_model(csa_model, data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=white_matter)

In [ ]:
from dipy.tracking.local import ThresholdTissueClassifier

classifier = ThresholdTissueClassifier(csa_peaks.gfa, .25)

In [ ]:
from dipy.tracking import utils

seed_mask = labels == 2
seeds = utils.seeds_from_mask(seed_mask, density=[2, 2, 2], affine=affine)

In [ ]:
from dipy.tracking.local import LocalTracking
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors
from dipy.tracking.streamline import Streamlines

# Initialization of LocalTracking. The computation happens in the next step.
streamlines_generator = LocalTracking(csa_peaks, classifier, seeds, affine, step_size=.5)

# Generate streamlines object
streamlines = Streamlines(streamlines_generator)

# Prepare the display objects.
color = line_colors(streamlines)

# Brayns

In [ ]:
import viztools
brayns = viztools.Brayns('bbpav03.bbp.epfl.ch:5000')

In [ ]:
def add_streamlines(radius):
    import math
    count = 0
    indices = []
    points = []
    i = 0
    for s in streamlines:
        indices.append(count)
        for p in s:
            points.append(p[0])
            points.append(p[1])
            points.append(p[2])
            
        count = count + len(s)
        i = i + 1
        
    params = dict()
    params['name'] = 'Deterministic'
    params['indices'] = indices
    params['points'] = points
    params['radius'] = radius
    params['opacity'] = 1
    result = brayns.rpc_request("streamline", params=params)

In [ ]:
add_streamlines(0.1)

In [ ]:
import math
streamline_id = 300
r = 10
nb_frames = len(streamlines[streamline_id])
print(nb_frames)
for i in range(nb_frames - 1):
    o = streamlines[streamline_id][i].tolist()
    t = streamlines[streamline_id][i+1].tolist()
    o[0] = o[0] + r * i / 10 * math.cos(i * math.pi / 180)
    o[2] = o[2] + r * i / 10 * math.sin(i * math.pi / 180)
    
#     v = [t[0] - o[0], t[1] - o[1], t[2] - o[2]]
#     o = [t[0] - r * v[0], t[1] - r * v[1], t[2] - r * v[2]]
    brayns.set_camera(origin=o, look_at=t, up=[0,0,1])

## Movie

In [ ]:
import math
r = 150
for i in range(360):
    angle = i * math.pi / 180.0
    t = [2.3650970458984375, -13.360580444335938, 24.217458724975586]
    o = [t[0] + r * math.cos(angle), t[1] + r * math.sin(angle), t[2]]
    brayns.set_camera(origin=o, look_at=t, up=[0,0,1])
    filename = '/home/favreau/medias/movies/dti/%06d.png' % i
    brayns.image(size=(1920,1080), samples_per_pixel=8, quality=100, format='png').save(filename)